In [1]:
from src.alg import solve_nd, sample, last
from src.alg import A as gen_A
import numpy as np
import numpy.linalg as LA

from matplotlib import cm
import matplotlib as mpl

mpl.use('pgf')

import matplotlib.pyplot as plt
plt.style.use('classic')


pgf_with_rc_fonts = {
    "font.family": "serif",
    "text.usetex": True,
    "pgf.texsystem": "xelatex",
    "pgf.rcfonts": False,
    "pgf.preamble": [
        r"\usepackage{unicode-math}",
        r"\setmainfont{EB Garamond}",
        r"\setmonofont{Courier New}",
        r"\setmathfont{Garamond-Math}",
    ]
}
mpl.rcParams.update(pgf_with_rc_fonts)

In [2]:
n = 10
k = 1/100


def F(x,y):
    return np.exp(-50 * ((x - 1/2)**2 + (y - 1/2)**2))

f = sample(F, n)

In [3]:
A = gen_A(k, n)

In [6]:
j     = list(solve_nd(A, f, method="J"))
gs    = list(solve_nd(A, f, method="GS"))
sor   = list(solve_nd(A, f, method="SOR", omega=0.5))
sor75 = list(solve_nd(A, f, method="SOR", omega=0.75))
sor25 = list(solve_nd(A, f, method="SOR", omega=0.25))

0.9594932134878021
0.9206272267291495
0.94659645454299
0.9362886398605412
0.9539547106489654


In [5]:
def right(it):
    for _, x in it: yield x

def relative_residual(appr):
    # we need a list, because we can't peek iterators
    assert type(appr) == list
    a = LA.norm(appr[0][1])
    for r in right(iter(appr)):
        yield LA.norm(r) / a

In [17]:
jrrs = list(relative_residual(j))
grrs = list(relative_residual(gs))
srrs = list(relative_residual(sor))
srrs25 = list(relative_residual(sor25))
srrs75 = list(relative_residual(sor75))

assert len(jrrs) == len(grrs) == len(srrs)

fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_yscale('log')

xs = list(range(len(jrrs)))

ax.plot(
    xs,
    jrrs,
    label="\scshape Jacobi", 
    color=cm.viridis(0.125),
)

ax.plot(
    xs,
    srrs25,
    label="\scshape S. O. R. $(\omega = 0.25)$", 
    color=cm.viridis(0.6),
)

ax.plot(
    xs,
    srrs,
    label="\scshape S. O. R. $(\omega = 0.50)$", 
    color=cm.viridis(0.7),
)


ax.plot(
    xs,
    srrs75,
    label="\scshape S. O. R. $(\omega = 0.75)$", 
    color=cm.viridis(0.8),
)

ax.plot(
    xs,
    grrs,
    label="\scshape Gauss-Seidel", 
    color=cm.viridis(0.875),
)

ax.set_xlabel("\scshape Iterations")
ax.set_ylabel("\scshape Residual")

ax.legend()

fig.savefig("residual.pdf")

In [13]:
from src.alg import sor_mat, spectral_radius


# A is already defined
def spec(omega):
    # find the spectral radius of C for S. O. R.
    # given omega.
    M, N = sor_mat(A, omega)
    Mi = LA.inv(M)
    C = Mi.dot(N)
    
    return omega, spectral_radius(C)


In [15]:
om = np.linspace(1, 2, 200)

rho = [spec(o) for o in om]

o, r = min(rho, key = lambda x: x[1])
o, r

(1.5628140703517588, 0.7198207809948826)

In [54]:
# benches
import time
from src.alg import solve_nd_prec

start = time.time()
for _ in range(1000):
    i, s = solve_nd(A, f, method="j", omega=1.5628, tol=1.0E-10)
end = time.time()

(end - start) * 1.0E6, i

(13718915.224075317, 552)